## Simulation 
We begin by specifying packages, reading in the dataset and generating some functions that we will use repeatedly to calculate poverty rates and gini coefficients as well as to generate simple percentage changes.

In the dataframe, we have spmu unit years as the level of the dataset. All values are averaged across 2018-2020.

In [1]:
import microdf as mdf
import pandas as pd
import numpy as np
import us
import plotly.express as px
from ubicenter import format_fig

person_sim = pd.read_csv(
    "jb/data/cps_00003.csv.gz",
    compression="gzip")

In [2]:
# Define a function to calculate poverty rates from the poverty flag
def pov(data, group):
    return pd.DataFrame(
        mdf.weighted_mean(data, "poverty_flag", "asecwt", groupby=group)
    )

def deep_pov(data, group):
    return pd.DataFrame(
        mdf.weighted_mean(data, "deep_poverty_flag", "asecwt", groupby=group)
)

# Define function to generate gini coefficients
def gin(data, group):
    return pd.DataFrame(
        data.groupby(group).apply(
            lambda x: mdf.gini(x, "spmtotres", "asecwt")
        )
    )

# Define percentage change function
def percent_change(new, old):
    return 100 * (new - old) / old

We generate poverty rates for the total population and by demographics of interest, namely sex, race, whether one is a child (under 6) and by state. We similarly generate gini coefficients for the total population and by state.

In [10]:
# Poverty rates by demographics of interest
poverty_rate = pov(person_sim, ["sim"]) # Overall poverty rate
poverty_rate_sex = pov(person_sim, ["sim", "sex"]) # Poverty rates by sex
poverty_rate_race_hispan = pov(person_sim, ["sim", "race_hispan"])  # Poverty rates by race
poverty_rate_child = pov(person_sim[person_sim.child_6], ["sim", "ca"]) # Child poverty rate

# State-based poverty rates
poverty_rate_state = pov(person_sim, ["sim", "state"])

# Rename constructed poverty_rates
poverty_rates = [
    poverty_rate,
    poverty_rate_sex,
    poverty_rate_race_hispan,
    poverty_rate_state,
    poverty_rate_child,
]
for i in poverty_rates:
    i.rename({0: "poverty_rate"}, axis=1, inplace=True)


,index,sim,poverty_rate
0,0,baseline,0.127830
1,1,high_ca,0.099814
2,2,high_xpns,0.099875
3,3,low_ca,0.105614
4,4,low_xpns,0.105880
5,5,replace_ca,0.120064
6,6,replace_xpns,0.125228


Index(['baseline', 'high_ca', 'high_xpns', 'low_ca', 'low_xpns', 'replace_ca',
       'replace_xpns'],
      dtype='object', name='sim')

In [18]:
   
# Gini coefficients and state/demographic-based heterogenous gini coefficients
gini = gin(person_sim, ["sim"])
gini_state = gin(person_sim, ["sim", "state"])

# Rename constructed gini coefficients
ginis = [
    gini,
    gini_state,
]
for i in ginis:
    i.rename({0: "gini_coefficient"}, axis=1, inplace=True)

# Create pivot table to interpret state-based poverty effects
state_pov = poverty_rate_state.pivot_table(
    values="poverty_rate", index="state", columns="sim"
)
# Create pivot table to interpret state-based gini effects
state_gini = gini_state.pivot_table(
    values="gini_coefficient", index="state", columns="sim"
)

We then generate state-based poverty rate and gini percentage changes to reflect the impact of the simulation. 

In [1]:
poverty_rate

NameError: name 'poverty_rate' is not defined

In [26]:

"""
# Generate state-based poverty rate percentage changes
state_pov["poverty_change_cc"] =  state_pov.cc_replacement - state_pov.baseline
state_pov["poverty_change_flat"] = state_pov.child_allowance - state_pov.baseline
state_pov["poverty_change_pc_cc"] = state_pov.poverty_change_cc - state_pov.baseline
state_pov["poverty_change_pc_flat"] = (
    state_pov.poverty_change_flat - state_pov.baseline
)

# Construct state-based gini coefficient percentage changes
state_gini["gini_change_cc"] = state_gini.cc_replacement - state_gini.baseline
state_gini["gini_change_flat"] = state_gini.child_allowance - state_gini.baseline
state_gini["gini_change_pc_cc"] = percent_change(
    state_gini.gini_change_cc, state_gini.baseline
)
state_gini["gini_change_pc_flat"] = percent_change(
    state_gini.gini_change_flat, state_gini.baseline
)

# Re-arrange and present pivot tables, descending by % change
# in poverty rate
state_pov.sort_values(by="poverty_change_pc_flat", ascending=True)
state_gini.sort_values(by="gini_change_pc_flat", ascending=True)
"""

'\n# Generate state-based poverty rate percentage changes\nstate_pov["poverty_change_cc"] =  state_pov.cc_replacement - state_pov.baseline\nstate_pov["poverty_change_flat"] = state_pov.child_allowance - state_pov.baseline\nstate_pov["poverty_change_pc_cc"] = state_pov.poverty_change_cc - state_pov.baseline\nstate_pov["poverty_change_pc_flat"] = (\n    state_pov.poverty_change_flat - state_pov.baseline\n)\n\n# Construct state-based gini coefficient percentage changes\nstate_gini["gini_change_cc"] = state_gini.cc_replacement - state_gini.baseline\nstate_gini["gini_change_flat"] = state_gini.child_allowance - state_gini.baseline\nstate_gini["gini_change_pc_cc"] = percent_change(\n    state_gini.gini_change_cc, state_gini.baseline\n)\nstate_gini["gini_change_pc_flat"] = percent_change(\n    state_gini.gini_change_flat, state_gini.baseline\n)\n\n# Re-arrange and present pivot tables, descending by % change\n# in poverty rate\nstate_pov.sort_values(by="poverty_change_pc_flat", ascending=True

## Visualizations


In [5]:
poverty_rate.reset_index(inplace=True)
poverty_rate.plot.bar(x="sim",y="poverty_rate",rot=70,title="Poverty rate by simulation")
# to png option in plotly

poverty_rate_child.plot.bar(x="sim",y="poverty_rate",rot=70,title="Poverty rate by simulation")

KeyError: 'sim'

In [ ]:
poverty_rate.reset_index(inplace=True)
fig = px.bar(poverty_rate,x="sim",y="poverty_rate")
format_fig(fig)

In [13]:
poverty_rate_child.reset_index(inplace=True)
fig = px.bar(poverty_rate_child,x="sim",y="poverty_rate")
format_fig(fig)

In [29]:
poverty_rate

,poverty_rate
sim,
baseline,0.127830
high_ca,0.000000
high_xpns,0.000000
low_ca,0.000000
low_xpns,0.000000
replace_ca,0.120064
replace_xpns,0.125228


## Takeup regression analysis